In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
data=pd.read_csv("Stars.csv")
data

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0
...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,Blue,O,5
236,30839,834042.000000,1194.0000,-10.63,Blue,O,5
237,8829,537493.000000,1423.0000,-10.73,White,A,5
238,9235,404940.000000,1112.0000,-11.23,White,A,5


In [3]:
Temperature=data['Temperature']
iqr=Temperature.quantile(0.75)-Temperature.quantile(0.25)
d=1.5*iqr
d1=Temperature.quantile(0.25)-d
d2=Temperature.quantile(0.75)+d
print(f"( {d1} , {d2} )")

( -14222.625 , 32622.375 )


In [4]:
print(max(Temperature))

40000


In [10]:
dataNoise =data.loc[data['Temperature']>d2].copy()
AnotherData=data.loc[data['Temperature']<d2].copy()
dataNoise

,Temperature,L,R,A_M,Color,Spectral_Class,Type
30,39000,204000.0,10.60,-4.70,Blue,O,3
49,33750,220000.0,26.00,-6.10,Blue,B,4
99,36108,198000.0,10.20,-4.40,Blue,O,3
100,33300,240000.0,12.00,-6.50,Blue,B,4
101,40000,813000.0,14.00,-6.23,Blue,O,4
109,33421,352000.0,67.00,-5.79,Blue,O,4
159,37800,202900.0,6.86,-4.56,Blue,O,3
214,34190,198200.0,6.39,-4.57,Blue,O,3
231,38234,272830.0,1356.00,-9.29,Blue,O,5
235,38940,374830.0,1356.00,-9.93,Blue,O,5


In [13]:
X=AnotherData[['L','R']]
reg=LinearRegression().fit(X,AnotherData['Temperature'])

In [19]:
predictNoise=reg.predict(dataNoise[['L','R']])
predictNoise

array([12198.98102923, 12467.00855375, 12078.76434074, 12923.94746098,
       24537.36130087, 14993.72081481, 12190.38440594, 12096.78905375,
        8662.24914374, 10730.86369354,  7544.39094696])

In [21]:
data['TemperatureNoise']=data['Temperature'].copy()
data['TemperatureNoise'].loc[data['Temperature']>d2]=predictNoise
data[['TemperatureNoise','Temperature']].loc[data['Temperature']>d2]

C:\Users\Mosp\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,TemperatureNoise,Temperature
30,12198.981029,39000
49,12467.008554,33750
99,12078.764341,36108
100,12923.947461,33300
101,24537.361301,40000
109,14993.720815,33421
159,12190.384406,37800
214,12096.789054,34190
231,8662.249144,38234
235,10730.863694,38940


In [ ]:
selectrandom=np.random.randint(0,240,size=10)
selectrandom

In [ ]:
newTemp=data['Temperature'].copy()
newTemp[list(selectrandom)]=np.nan
data['newTemp']=newTemp
data

In [ ]:
data2=data.loc[data['newTemp']==data['Temperature']]

In [ ]:
data2

In [ ]:
LR=data2[['L','R']]
np.array([np.array(LR['L']),np.array(LR['R'])])
arr=np.array([[x,y]for x,y in zip(np.array(LR['L']),np.array(LR['R']))])
arr

In [ ]:
reg=LinearRegression().fit(arr,data2['newTemp'])

In [ ]:
data4=data.loc[data['newTemp']!=data['Temperature']]
data4

In [ ]:
reg.predict([np.array([174000.00000,1284.0000])])

In [ ]:
data4LR=data4[['L','R']]
arr4=np.array([[x,y]for x,y in zip(np.array(data4LR['L']),np.array(data4LR['R']))])
data4['newTemp']=reg.predict(arr4)

In [ ]:
data4

In [ ]:
data['newTemp'].loc[data['Temperature']!=data['newTemp']]=data4['newTemp']

In [ ]:
data[['Temperature','newTemp']].loc[data['Temperature']!=data['newTemp']]

In [ ]:
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder


In [ ]:
color=data['Color']
lbl = LabelEncoder()
color=lbl.fit_transform(color)

Spectral_Class=data['Spectral_Class']
lbl = LabelEncoder()
Spectral_Class=lbl.fit_transform(Spectral_Class)


In [ ]:
df=(len(np.unique(Spectral_Class))-1)*(len(np.unique(color))-1)
df

In [ ]:
x_squared, p, dof, expected = chi2_contingency([Spectral_Class,color])
x_squared

In [ ]:
from scipy.stats import chi2
prob=0.95
critical = chi2.ppf(prob, df)
critical